## OntoUML templating

# K-means Clustering Evaluation for Embeddings

Now we'll implement a function to evaluate the classification performance of the embeddings using K-means clustering. We'll compare:
1. Ground truth CM embeddings vs. true class labels
2. Generated CM embeddings (from NL embeddings) vs. true class labels

In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, homogeneity_score
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import Counter

# Load the dataframe with class labels
with open("datasets/ontouml_nl2cm_df.pkl", "rb") as f:
    df = pickle.load(f)

# Load embeddings
x_path = "datasets/desc.npy"
y_path = "datasets/model.npy"
X = np.stack(df['NL_Serialization_Emb'])  # NL embeddings
Y = np.stack(df['CM_Serialization_Emb'])  # CM embeddings (ground truth)

# Normalize embeddings
X = X / (np.linalg.norm(X, axis=1, keepdims=True) + 1e-8)
Y = Y / (np.linalg.norm(Y, axis=1, keepdims=True) + 1e-8)

# Get class labels
class_labels = df['cls'].values
le = LabelEncoder()
class_labels_encoded = le.fit_transform(class_labels)
n_clusters = len(np.unique(class_labels_encoded))

print(f"Number of samples: {len(class_labels)}")
print(f"Number of unique classes: {n_clusters}")
print(f"Class distribution: {Counter(class_labels)}")

Number of samples: 190
Number of unique classes: 16
Class distribution: Counter({'H': 65, 'T': 52, 'Q': 13, 'K': 10, 'R': 10, 'G': 8, 'L': 7, 'B': 6, 'J': 5, 'Z': 4, 'S': 3, 'U': 2, 'A': 2, 'V': 1, 'C': 1, 'M': 1})


In [ ]:
def evaluate_with_kmeans(embeddings, true_labels, n_clusters):
    """
    Evaluate embeddings using K-means clustering and compare to true labels
    
    Args:
        embeddings: Normalized embedding vectors
        true_labels: Ground truth class labels (encoded)
        n_clusters: Number of clusters to create
        
    Returns:
        Dictionary of metrics
    """
    # Cluster the embeddings
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    predicted_clusters = kmeans.fit_predict(embeddings)
    
    # Calculate clustering metrics
    metrics = {
        "adjusted_rand_score": adjusted_rand_score(true_labels, predicted_clusters),
        "normalized_mutual_info": normalized_mutual_info_score(true_labels, predicted_clusters),
        "homogeneity_score": homogeneity_score(true_labels, predicted_clusters)
    }
    
    # Add intrinsic clustering metrics if the number of samples is sufficient
    if len(embeddings) > n_clusters:
        try:
            metrics["silhouette_score"] = silhouette_score(embeddings, predicted_clusters)
            metrics["davies_bouldin_score"] = davies_bouldin_score(embeddings, predicted_clusters)
        except Exception as e:
            print(f"Error calculating intrinsic metrics: {e}")
    
    return metrics, predicted_clusters

In [ ]:
from vec2vec import run as train_vec2vec
from vec_transform import run as train_vecmap
import os


def train_model(X, Y, model_type='vec2vec'):
    """
    Train a model to map NL embeddings to CM embeddings
    
    Args:
        X: Normalized NL embeddings
        Y: Normalized CM embeddings
        model_type: 'vec2vec' or 'vecmap' to specify which model to train
        
    Returns:
        None (saves the trained model)
    """
    assert model_type in ['vec2vec', 'vecmap'], "model_type must be 'vec2vec' or 'vecmap'"
    
    if model_type == 'vec2vec':
        train_vec2vec(
            X=X, Y=Y,
            batch_size=64, epochs=100, lr=1e-4,
            d_lat=256, T=4, heads=4, layers=2, mem_tokens=4, dropout=0.1,
            save_path="runs/vec2vec_xattn.pt"
        )
    elif model_type == 'vecmap':
        train_vecmap(
            X=X, Y=Y,
            batch_size=64, epochs=100, lr=1e-4,
            d_lat=512, T_src=8, T_tgt=4,
            n_enc=3, n_dec=3, heads=8, mlp_ratio=4.0, dropout=0.1,
            save_path="runs/vecmap_transformer.pt"
        )


def get_predicted_embeddings(model_type, nl_embeddings):
    """
    Generate predicted CM embeddings from NL embeddings using trained models
    
    Args:
        model_type: 'vec2vec' or 'vecmap' to specify which model to use
        nl_embeddings: Normalized NL embeddings
        
    Returns:
        Predicted CM embeddings
    """
    assert model_type in ['vec2vec', 'vecmap'], "model_type must be 'vec2vec' or 'vecmap'"
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    if model_type == 'vec2vec':
        # Load the vec2vec model
        assert os.path.exists("runs/vec2vec_xattn.pt"), "Trained vec2vec model not found at runs/vec2vec_xattn.pt"
        from vec2vec import Vec2VecXAttn
        
        d_text = nl_embeddings.shape[1]
        d_model = d_text  # Assuming same dimensions
        
        model = Vec2VecXAttn(
            d_text=d_text, 
            d_model=d_model,
            d_lat=256, T=4, heads=4, layers=2, mem_tokens=4, dropout=0.1
        ).to(device)
        
        model.load_state_dict(torch.load("runs/vec2vec_xattn.pt", map_location=device))
        model.eval()
        
        with torch.no_grad():
            x = torch.tensor(nl_embeddings, dtype=torch.float32, device=device)
            y_pred, _ = model.trans.text_to_model(x, y_hint=None)
            y_pred = F.normalize(y_pred, dim=-1).cpu().numpy()
        
    elif model_type == 'vecmap':
        # Load the vecmap model
        assert os.path.exists("runs/vecmap_transformer.pt"), "Trained vecmap model not found at runs/vecmap_transformer.pt"
        from vec_transform import VecMapTransformer
        
        d_src = nl_embeddings.shape[1]
        d_tgt = d_src  # Assuming same dimensions
        
        model = VecMapTransformer(
            d_src=d_src, d_tgt=d_tgt,
            d_lat=512, T_src=8, T_tgt=4,
            n_enc=3, n_dec=3, heads=8, mlp_ratio=4.0, dropout=0.1
        ).to(device)
        
        model.load_state_dict(torch.load("runs/vecmap_transformer.pt", map_location=device))
        model.eval()
        
        with torch.no_grad():
            x = torch.tensor(nl_embeddings, dtype=torch.float32, device=device)
            y_pred = model(x)
            y_pred = F.normalize(y_pred, dim=-1).cpu().numpy()
    else:
        raise ValueError(f"Unknown model type: {model_type}")
    
    return y_pred

In [ ]:
# First, evaluate ground truth CM embeddings with K-means
print("Evaluating ground truth CM embeddings...")
cm_metrics, cm_clusters = evaluate_with_kmeans(Y, class_labels_encoded, n_clusters)
print("Ground truth CM embeddings results:")
for metric, value in cm_metrics.items():
    print(f"  {metric}: {value:.4f}")

# Generate predicted embeddings using vec2vec model
try:
    print("\nGenerating vec2vec predicted embeddings...")
    vec2vec_pred = get_predicted_embeddings('vec2vec', X)
    
    # Evaluate vec2vec predicted embeddings
    print("Evaluating vec2vec predicted embeddings...")
    vec2vec_metrics, vec2vec_clusters = evaluate_with_kmeans(vec2vec_pred, class_labels_encoded, n_clusters)
    print("vec2vec predicted embeddings results:")
    for metric, value in vec2vec_metrics.items():
        print(f"  {metric}: {value:.4f}")
except Exception as e:
    print(f"Error evaluating vec2vec model: {e}")

# Generate predicted embeddings using vecmap model
try:
    print("\nGenerating vecmap predicted embeddings...")
    vecmap_pred = get_predicted_embeddings('vecmap', X)
    
    # Evaluate vecmap predicted embeddings
    print("Evaluating vecmap predicted embeddings...")
    vecmap_metrics, vecmap_clusters = evaluate_with_kmeans(vecmap_pred, class_labels_encoded, n_clusters)
    print("vecmap predicted embeddings results:")
    for metric, value in vecmap_metrics.items():
        print(f"  {metric}: {value:.4f}")
except Exception as e:
    print(f"Error evaluating vecmap model: {e}")

Evaluating ground truth CM embeddings...
Ground truth CM embeddings results:
  adjusted_rand_score: 0.0446
  normalized_mutual_info: 0.2797
  homogeneity_score: 0.3206
  silhouette_score: 0.0406
  davies_bouldin_score: 2.9012

Generating vec2vec predicted embeddings...
Evaluating vec2vec predicted embeddings...
vec2vec predicted embeddings results:
  adjusted_rand_score: 0.0789
  normalized_mutual_info: 0.3207
  homogeneity_score: 0.3669
  silhouette_score: 0.0386
  davies_bouldin_score: 2.7136

Generating vecmap predicted embeddings...
Evaluating vecmap predicted embeddings...
vecmap predicted embeddings results:
  adjusted_rand_score: 0.0543
  normalized_mutual_info: 0.3349
  homogeneity_score: 0.3940
  silhouette_score: 0.1031
  davies_bouldin_score: 2.0383


In [ ]:
n_clusters

16

In [ ]:
# Visualize the comparison of metrics
def plot_metrics_comparison(cm_metrics, vec2vec_metrics=None, vecmap_metrics=None):
    # Collect metrics for comparison
    metrics = list(cm_metrics.keys())
    values = {
        'Ground Truth CM': [cm_metrics[m] for m in metrics]
    }
    
    if vec2vec_metrics is not None:
        values['vec2vec Predicted'] = [vec2vec_metrics.get(m, 0) for m in metrics]
    
    if vecmap_metrics is not None:
        values['vecmap Predicted'] = [vecmap_metrics.get(m, 0) for m in metrics]
    
    # Plot
    plt.figure(figsize=(10, 6))
    x = np.arange(len(metrics))
    width = 0.2
    multiplier = 0
    
    for model, scores in values.items():
        offset = width * multiplier
        rects = plt.bar(x + offset, scores, width, label=model)
        multiplier += 1
    
    plt.ylabel('Score')
    plt.title('Comparison of Clustering Metrics')
    plt.xticks(x + width, metrics, rotation=45)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3)
    plt.tight_layout()
    
    # Add value annotations on top of bars
    for i, model in enumerate(values.keys()):
        for j, v in enumerate(values[model]):
            plt.text(j + width * (i - 0.5), v + 0.01, f'{v:.2f}', 
                     ha='center', va='bottom', fontsize=8)
    
    plt.ylim(0, 1.0)  # All metrics are between 0 and 1
    plt.show()

try:
    # Create the comparison visualization
    plot_metrics_comparison(
        cm_metrics, 
        vec2vec_metrics if 'vec2vec_metrics' in locals() else None,
        vecmap_metrics if 'vecmap_metrics' in locals() else None
    )
except Exception as e:
    print(f"Error creating visualization: {e}")

In [ ]:
# Visualize the clusters using t-SNE
from sklearn.manifold import TSNE

def plot_tsne_clusters(embeddings, labels, title):
    # Apply t-SNE dimensionality reduction
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(embeddings)-1))
    reduced_data = tsne.fit_transform(embeddings)
    
    # Create a scatter plot
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=labels, 
                          cmap='tab20', alpha=0.8, s=50)
    
    # Add a legend with class names
    class_names = [f"Class {i}" for i in range(n_clusters)]
    legend1 = plt.legend(handles=scatter.legend_elements()[0], 
                        labels=class_names,
                        title="Classes", loc="upper right")
    plt.gca().add_artist(legend1)
    
    plt.title(title)
    plt.tight_layout()
    plt.show()

# Visualize clusters for all embeddings
try:
    print("\nVisualizing clusters using t-SNE...")
    plot_tsne_clusters(Y, class_labels_encoded, "Ground Truth CM Embeddings")
    
    if 'vec2vec_pred' in locals():
        plot_tsne_clusters(vec2vec_pred, class_labels_encoded, "vec2vec Predicted CM Embeddings")
    
    if 'vecmap_pred' in locals():
        plot_tsne_clusters(vecmap_pred, class_labels_encoded, "vecmap Predicted CM Embeddings")
except Exception as e:
    print(f"Error creating t-SNE visualizations: {e}")

In [ ]:
from serialization import serialize_archimate_models

serialize_archimate_models('datasets')

In [ ]:
from embed import add_embeddings
import pandas as pd

add_embeddings(
    pd.read_csv('datasets/eamodelset_serialized.csv'), 
    embedding_columns=['NL_Serialization', 'CM_Serialization'], 
    num_jobs=8, 
    output_pth='datasets/eamodelset_nl2cm_embeddings_df'
)

Processing column: NL_Serialization


Generating embeddings:   0%|          | 0/978 [00:00<?, ?it/s]

Processing column: CM_Serialization


Generating embeddings:   0%|          | 0/978 [00:00<?, ?it/s]

In [31]:
import pickle

with open("datasets/eamodelset_nl2cm_embeddings_df.pkl", "rb") as f:
    df = pickle.load(f)

df.head()

,key,NL_Serialization,CM_Serialization,NL_Serialization_Emb,CM_Serialization_Emb
0,id-85903050004a40d8964c6edc97e82d57,Elements: \n\tZenodo\n\tBonaRes\n\tEUSO\n\tESD...,Elements: \n\tAn ArchiMate ApplicationComponen...,"[-0.0031474686693400145, 0.027788767591118813,...","[-0.017474092543125153, 0.03106505423784256, 0..."
1,id-55d5ef7a83cc49bb8ed47978945fbdbc,Elements: \n\tcustomer\n\tBusiness Interface\n...,Elements: \n\tAn ArchiMate BusinessRole elemen...,"[0.023898378014564514, 0.025850150734186172, 0...","[0.0024201564956456423, 0.03527628257870674, 0..."
2,cc8dfbb0-a179-4ee3-9555-a67292544a96,Elements: \n\tRessource\n\tKapabilitet \n\tKap...,Elements: \n\tAn ArchiMate Resource element na...,"[-0.05094992741942406, 0.05218298360705376, 0....","[-0.035470303148031235, 0.04340995103120804, 0..."
3,3586dd56-a7ec-496a-a1c0-4633ed13b319,Elements: \n\tGC Enterprise Architecture Frame...,Elements: \n\tAn ArchiMate Resource element na...,"[0.03228999674320221, 0.005926018580794334, 0....","[0.01876358687877655, 0.00034540516207925975, ..."
4,1f0d7762-0011-48d2-a572-f535f1e5a423,Elements: \n\tOnboard Host Department\n\tSSC E...,Elements: \n\tAn ArchiMate BusinessEvent eleme...,"[0.005651684477925301, 0.003968433942645788, 0...","[-0.009953605011105537, 0.00875463243573904, 0..."


In [33]:
X = df['NL_Serialization_Emb']

In [34]:
# kmeans_model_selection.py
import math
import numpy as np
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score, adjusted_rand_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state


@dataclass
class KMetrics:
    k: int
    inertia: float
    silhouette: Optional[float]
    calinski_harabasz: Optional[float]
    davies_bouldin: Optional[float]
    gap: Optional[float]
    gap_se: Optional[float]
    stability_ari_median: Optional[float]


def _elbow_knee(x: List[int], y: List[float]) -> Optional[int]:
    """
    Elbow via 'maximum distance to baseline' (triangle) method.
    x: ks, y: SSE/inertia (monotonically decreasing)
    Returns the k (from x) where the curvature is largest.
    """
    if len(x) < 3:
        return None
    x_arr, y_arr = np.asarray(x), np.asarray(y)
    # line from first to last
    x1, y1 = x_arr[0], y_arr[0]
    x2, y2 = x_arr[-1], y_arr[-1]
    # distances of points to the line
    denom = math.hypot(x2 - x1, y2 - y1)
    if denom == 0:
        return None
    dists = []
    for xi, yi in zip(x_arr, y_arr):
        num = abs((y2 - y1) * xi - (x2 - x1) * yi + x2*y1 - y2*x1)
        dists.append(num / denom)
    idx = int(np.argmax(dists))
    return int(x_arr[idx])


def _gap_statistic(X: np.ndarray, k: int, B: int = 10, random_state: Optional[int] = 42,
                   sample_n: Optional[int] = None) -> Tuple[float, float]:
    """
    Tibshirani et al. (2001): Gap(k) = E*_b[log(W*_k)] - log(W_k)
    where W_k is within-cluster dispersion. Reference data uniform in bounding box.
    Returns (gap, se_gap).
    """
    rng = check_random_state(random_state)
    n, d = X.shape

    if sample_n is not None and sample_n < n:
        idx = rng.choice(n, size=sample_n, replace=False)
        X_used = X[idx]
    else:
        X_used = X

    mins = X_used.min(axis=0)
    maxs = X_used.max(axis=0)

    # Fit on real data
    kmeans = KMeans(n_clusters=k, n_init=10, init="k-means++", random_state=random_state)
    labels = kmeans.fit_predict(X_used)
    Wk = _within_cluster_dispersion(X_used, labels)

    # Reference dispersions
    log_Wk_star = []
    for b in range(B):
        X_ref = rng.uniform(mins, maxs, size=(X_used.shape[0], d))
        km_ref = KMeans(n_clusters=k, n_init=10, init="k-means++", random_state=rng.randint(0, 10**9))
        labels_ref = km_ref.fit_predict(X_ref)
        Wk_star = _within_cluster_dispersion(X_ref, labels_ref)
        log_Wk_star.append(np.log(Wk_star))

    log_Wk_star = np.asarray(log_Wk_star)
    gap = log_Wk_star.mean() - np.log(Wk)
    se = np.sqrt(1 + 1.0 / B) * log_Wk_star.std(ddof=1)
    return float(gap), float(se)


def _within_cluster_dispersion(X: np.ndarray, labels: np.ndarray) -> float:
    """Sum of squared distances to cluster centroids (like inertia but recomputed to use with any labels)."""
    W = 0.0
    for k in np.unique(labels):
        Xk = X[labels == k]
        if len(Xk) == 0:
            continue
        ck = Xk.mean(axis=0, keepdims=True)
        W += ((Xk - ck) ** 2).sum()
    return float(W)


def _stability_ari(X: np.ndarray, k: int, R: int = 10, random_state: Optional[int] = 42) -> float:
    """
    Stability via multiple runs with different seeds on the SAME data.
    Returns median pairwise ARI across R runs.
    """
    rng = check_random_state(random_state)
    labelings = []
    seeds = rng.randint(0, 10**9, size=R)
    for s in seeds:
        km = KMeans(n_clusters=k, n_init=10, init="k-means++", random_state=int(s))
        labels = km.fit_predict(X)
        labelings.append(labels)
    # pairwise ARI
    if len(labelings) < 2:
        return np.nan
    aris = []
    for i in range(len(labelings)):
        for j in range(i + 1, len(labelings)):
            aris.append(adjusted_rand_score(labelings[i], labelings[j]))
    return float(np.median(aris))


def evaluate_k_range(
    X: np.ndarray,
    ks: Optional[List[int]] = None,
    scale: bool = False,
    gap_B: int = 10,
    gap_sample_n: Optional[int] = None,
    stability_R: int = 10,
    random_state: Optional[int] = 42,
) -> Dict[int, KMetrics]:
    """
    Compute metrics for a range of k.
    """
    if scale:
        X_proc = StandardScaler().fit_transform(X)
    else:
        X_proc = X

    n = X_proc.shape[0]
    if ks is None:
        kmax = int(min(10, max(3, 2 * math.sqrt(n))))
        ks = list(range(2, max(3, kmax) + 1))

    results: Dict[int, KMetrics] = {}

    for k in ks:
        if k >= n:
            # skip degenerate ks
            continue
        km = KMeans(n_clusters=k, n_init=10, init="k-means++", random_state=random_state)
        labels = km.fit_predict(X_proc)

        inertia = float(km.inertia_)
        # Some metrics require >1 sample per cluster; guard against failures
        try:
            sil = float(silhouette_score(X_proc, labels))
        except Exception:
            sil = np.nan
        try:
            ch = float(calinski_harabasz_score(X_proc, labels))
        except Exception:
            ch = np.nan
        try:
            db = float(davies_bouldin_score(X_proc, labels))
        except Exception:
            db = np.nan

        try:
            gap, gap_se = _gap_statistic(X_proc, k, B=gap_B, random_state=random_state, sample_n=gap_sample_n)
        except Exception:
            gap, gap_se = np.nan, np.nan

        try:
            stab = _stability_ari(X_proc, k, R=stability_R, random_state=random_state)
        except Exception:
            stab = np.nan

        results[k] = KMetrics(
            k=k,
            inertia=inertia,
            silhouette=sil,
            calinski_harabasz=ch,
            davies_bouldin=db,
            gap=gap,
            gap_se=gap_se,
            stability_ari_median=stab,
        )

    return results


def pick_k(results: Dict[int, KMetrics]) -> Dict[str, object]:
    """
    Aggregate ranks across metrics to pick best k.
    - Inertia (lower is better)
    - Silhouette (higher)
    - Calinski–Harabasz (higher)
    - Davies–Bouldin (lower)
    - Gap (higher) with 1-SE rule optional
    - Stability ARI (higher)

    Returns dict with chosen k, per-metric winners, ranks, and elbow_k.
    """
    ks = sorted(results.keys())
    if not ks:
        raise ValueError("No valid k evaluated.")

    # Collect arrays
    inertia = np.array([results[k].inertia for k in ks])
    sil = np.array([results[k].silhouette for k in ks])
    ch = np.array([results[k].calinski_harabasz for k in ks])
    db = np.array([results[k].davies_bouldin for k in ks])
    gap = np.array([results[k].gap for k in ks])
    stab = np.array([results[k].stability_ari_median for k in ks])

    # Elbow on inertia
    elbow_k = _elbow_knee(ks, inertia.tolist())

    # 1-SE rule for Gap (choose smallest k with gap >= gap[k+1] - se[k+1])
    gap_se = np.array([results[k].gap_se for k in ks])
    gap_k = None
    if np.isfinite(gap).any():
        # default: argmax gap
        gap_k = ks[int(np.nanargmax(gap))]
        # 1-SE rule refinement
        try:
            idx_best = int(np.nanargmax(gap))
            gap_best = gap[idx_best]
            se_best = gap_se[idx_best] if np.isfinite(gap_se[idx_best]) else 0.0
            # pick smallest k where gap >= gap_best - se_best
            for i, k in enumerate(ks):
                if np.isfinite(gap[i]) and gap[i] >= (gap_best - se_best):
                    gap_k = k
                    break
        except Exception:
            pass

    # Metric-wise best ks
    best = {
        "inertia": ks[int(np.nanargmin(inertia))],
        "silhouette": ks[int(np.nanargmax(sil))] if np.isfinite(sil).any() else None,
        "calinski_harabasz": ks[int(np.nanargmax(ch))] if np.isfinite(ch).any() else None,
        "davies_bouldin": ks[int(np.nanargmin(db))] if np.isfinite(db).any() else None,
        "gap": gap_k,
        "stability": ks[int(np.nanargmax(stab))] if np.isfinite(stab).any() else None,
        "elbow": elbow_k,
    }

    # Rank aggregation (lower is better rank)
    def rank(arr: np.ndarray, higher_is_better: bool) -> np.ndarray:
        vals = arr.copy()
        # handle nan by placing them worst
        if higher_is_better:
            order = np.argsort(np.argsort(np.nan_to_num(-vals, nan=-np.inf)))
        else:
            order = np.argsort(np.argsort(np.nan_to_num(vals, nan=np.inf)))
        # Convert to ranks starting at 1
        ranks = np.empty_like(order, dtype=float)
        ranks[order] = np.arange(1, len(order) + 1)
        # Inflate ranks for nans to worst
        ranks[~np.isfinite(arr)] = len(order) + 1
        return ranks

    ranks = []
    ranks.append(rank(inertia, higher_is_better=False))
    ranks.append(rank(sil, higher_is_better=True))
    ranks.append(rank(ch, higher_is_better=True))
    ranks.append(rank(db, higher_is_better=False))
    ranks.append(rank(gap, higher_is_better=True))
    ranks.append(rank(stab, higher_is_better=True))

    mean_rank = np.vstack(ranks).mean(axis=0)
    agg_best_k = ks[int(np.argmin(mean_rank))]

    return {
        "best_k": int(agg_best_k),
        "mean_rank": {k: float(r) for k, r in zip(ks, mean_rank)},
        "per_metric_winner": best,
        "elbow_k": elbow_k,
    }


def cluster_with_best_k(
    X: np.ndarray,
    ks: Optional[List[int]] = None,
    scale: bool = False,
    gap_B: int = 10,
    gap_sample_n: Optional[int] = None,
    stability_R: int = 10,
    random_state: Optional[int] = 42,
) -> Tuple[int, np.ndarray, KMeans, Dict[int, KMetrics], Dict[str, object]]:
    """
    Full pipeline: evaluate -> pick k -> fit final KMeans.
    Returns: (best_k, labels, model, all_results, selection_info)
    """
    results = evaluate_k_range(
        X, ks=ks, scale=scale, gap_B=gap_B, gap_sample_n=gap_sample_n,
        stability_R=stability_R, random_state=random_state
    )
    selection = pick_k(results)
    best_k = selection["best_k"]

    # Final fit (use scale choice consistent with evaluation)
    X_proc = StandardScaler().fit_transform(X) if scale else X
    model = KMeans(n_clusters=best_k, n_init=50, init="k-means++", random_state=random_state)
    labels = model.fit_predict(X_proc)
    return best_k, labels, model, results, selection


# ---------------------- Example usage ----------------------
# if __name__ == "__main__":
# Example: random embeddings; replace with your own X (n_samples x n_dims)
rng = np.random.default_rng(0)
# make 3 clusters in 128-D
X = np.vstack([
    rng.normal(0, 1, size=(300, 128)),
    rng.normal(4, 1, size=(300, 128)),
    rng.normal(-3, 1.2, size=(300, 128)),
])

best_k, labels, model, results, selection = cluster_with_best_k(
    X,
    ks=None,            # auto range: 2..min(10, 2*sqrt(n))
    scale=False,        # embeddings often pre-normalized; set True if needed
    gap_B=10,           # increase for more robust gap stat (slower)
    gap_sample_n=800,   # sample for gap to speed up on big n
    stability_R=8,
    random_state=42,
)

print("Best k:", best_k)
print("Per-metric winners:", selection["per_metric_winner"])
print("Elbow k (inertia):", selection["elbow_k"])
print("Mean rank per k:", selection["mean_rank"])


Best k: 2
Per-metric winners: {'inertia': 10, 'silhouette': 2, 'calinski_harabasz': 3, 'davies_bouldin': 2, 'gap': 3, 'stability': 2, 'elbow': 3}
Elbow k (inertia): 3
Mean rank per k: {2: 2.6666666666666665, 3: 3.0, 4: 3.8333333333333335, 5: 4.5, 6: 5.166666666666667, 7: 6.666666666666667, 8: 6.333333333333333, 9: 7.166666666666667, 10: 5.666666666666667}
